# Load data

In [2]:
# Load openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter

In [3]:
import pandas as pd

existing_file_path = 'data/input/test_input.xlsx'
# existing_file_path = 'data/input/test_input_minimized.xlsx'
# existing_file_path = 'data/input/test_data_table.xlsx'
# existing_file_path = 'data/input/tc_pride_data.xlsx'
new_file_path = 'data/output/test_output.xlsx'


# Start reading the data from the second row
df = pd.read_excel(existing_file_path, header=1)

# df = pd.read_excel(existing_file_path)

# Reading data with pandas

In [4]:
# Read the entire dataframe
# df

## Read headers
# print(df.columns)

## Read individual columns
# print(df.Date)
#  or
# print(df['Net Donation'])
#  or
# print(df[['Date', "Description", "Net Donation"]])

## Read top 3 rows
# print(df.head(3))

## Read row 3 (which has an index of 2),
# print(df.iloc[2])
# then 1-5 (which is index 0-4)
# print(df.iloc[0:5])
# Read specific location (R, C)
# print(df.iloc[2, 1]) # 3rd row, 2nd column

## Iterate over rows
# for index, row in df.iterrows():
    # print(index, row['Date'], row['Description'])
    # print(index, row)
    # print(index, row.Date)

# Conditional selection of rows
# df.loc[df['Net Donation'] > 60]

# Generate statistics
# df.describe()


# Sorting data

In [5]:
# df # Before sorting

# Sorting by single columns
# df = df.sort_values(by=['Description'])
# df = df.sort_values(by=['Event'])
# df = df.sort_values(by=['Source Title'])

# Sorting can be done on multiple columns with this one line of code
df = df.sort_values(by=['Source Title','Event','Description'])

# print(df)
# df # After sorting

# Sum revenue into different categories

In [6]:

def categorize_revenue(description, event, source_title):
  category = 'unknown'
  # determine which category the row belongs to
  # Pride festival
  # it's the festival if the event contains 'Twin Cities Pride Festival'
  if 'twin cities pride festival' in event and ('pride march' not in source_title and 'book fair' not in source_title):
    category = 'Pride Festival'

  # Pride parade
  # It's a parade if the event contains 'Twin Cities Pride Festival' and source title contains "TC Pride March Application"
  if 'twin cities pride festival' in event and 'pride march' in source_title:
    category = 'Pride Parade'
  
  # Book fair
  # book fair is in the event or source title
  if 'book fair' in event or 'book fair' in source_title:
    category = 'Book Fair'

  # Donation
  # if 'subscription' is in the description, it's a donation
  if 'subscription' in description or 'donation' in description:
    category = 'Donation'
  
  # Merchandise
  if 'twin cities pride - order' in description:
    category = 'Merchandise'
  
  return category


def check_refund(description, net_donation):
  if ('refund' in description or 'return' in description) and net_donation < 0:
    return True
  else:
    return False

def safe_lower(input):
    if isinstance(input, str):  # Checks if input is a string
        return input.lower()
    else:
        return '' # If not a string, return an empty string

def safe_number(input):
    if isinstance(input, (int, float)):  # Checks if input is a number
        return input
    else:
        return 0 # If not a number, return 0

# Variables
category_map = {}
total_donations = 0

# OPTIONAL - Set the preferred date format
preferred_date_format = 'string'
# OPTIONAL - Set the preferred date format

for index, row in df.iterrows():
  description = safe_lower(row['Description'])
  event = safe_lower(row['Event'])
  source_title = safe_lower(row['Source Title'])
  net_donation = safe_number(row['Net Donation'])
  
  # Categorize
  category = categorize_revenue(description, event, source_title)
  # Add it to the dataframe
  df.at[index, 'Category'] = category
  
  
  # OPTIONAL: Convert the Date column to a string
  # If the Date column is a datetime object, convert it to a string
  if preferred_date_format == 'string' and isinstance(row['Date'], pd.Timestamp):
    df.at[index, 'Date'] = row['Date'].strftime('%m/%d/%Y')
  elif preferred_date_format == 'datetime' and isinstance(row['Date'], str):
    df.at[index, 'Date'] = pd.to_datetime(row['Date'])
  # OPTIONAL: Convert the Date column to a string
  
  
  # Check for refund
  is_refund = check_refund(description, net_donation)
  
  # Add category to map
  if category not in category_map:
    category_map[category] = 0

  # Sum up donations
  category_map[category] += net_donation
  total_donations += net_donation
  
  # print('category is:', category, '. refund?:', is_refund)
  # Check the description, event, and source against a dictionary of keywords

# print(category_map)
# print('Total donations:', total_donations)


# Convert the dataframe data back into its original form


In [7]:
# Date conversions might be needed for writing to excel files
# df['Date'] = pd.to_datetime(df['Date'])

# Turn the date into a string
# df.at[index, 'Date (as string)'] = row['Date'].strftime('%m/%d/%Y')
# df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')

# Openpyxl to write data into a template file

In [8]:
# This uses openpyxl to load the pre-formatted Excel template and insert the data from the DataFrame into the correct columns
"""

# Load the pre-formatted Excel template
wb = load_workbook('data/output/test_template.xlsx')
ws = wb.active


# Need to get the column names in order to input all the data in the correct columns
# Get the column names from dataframe as an Index object
column_names = df.columns

# Define starting cell row and column for data insertion
start_row = 2
start_col = 1

# Insert DataFrame data into Excel based on column names
for index, row in df.iterrows():
    for col_index, column_name in enumerate(column_names):
        cell = ws.cell(row=index + start_row, column=col_index + start_col, value=row[column_name])


# Save the workbook as the final output
wb.save('data/output/final_output_from_template.xlsx')"""

"\n\n# Load the pre-formatted Excel template\nwb = load_workbook('data/output/test_template.xlsx')\nws = wb.active\n\n\n# Need to get the column names in order to input all the data in the correct columns\n# Get the column names from dataframe as an Index object\ncolumn_names = df.columns\n\n# Define starting cell row and column for data insertion\nstart_row = 2\nstart_col = 1\n\n# Insert DataFrame data into Excel based on column names\nfor index, row in df.iterrows():\n    for col_index, column_name in enumerate(column_names):\n        cell = ws.cell(row=index + start_row, column=col_index + start_col, value=row[column_name])\n\n\n# Save the workbook as the final output\nwb.save('data/output/final_output_from_template.xlsx')"

# Pandas writing to a new Excel sheet

In [9]:
# Pandas method to save the dataframe to an excel file
df.to_excel(new_file_path, index=False)

# Alternative saving method, this tries to preserve date column format using Pandas
# with pd.ExcelWriter('output.xlsx', date_format='mm/dd/yyyy hh:mm:ss', datetime_format='mm/dd/yyyy hh:mm:ss') as writer:
#     df.to_excel(writer, index=False)

# Use Openpyxl to make formatting changes

### Use Openpyxl to save the format of each column before they're overwritten

In [10]:
# This logic is not needed
"""# Load the original workbook and active sheet
wb = load_workbook(existing_file_path)
ws = wb.active

# Store formatting in a dictionary
original_formats = []
# Get the second row and iterate through each cell
row = ws[2]
for cell in row:
    # Push the column and the current format to the dictionary array
    original_formats.append((cell.column, cell.number_format))

print(original_formats)"""

[(1, 'General'), (2, 'General'), (3, 'General'), (4, 'General'), (5, 'General'), (6, 'General'), (7, 'General'), (8, 'General'), (9, 'General'), (10, 'General'), (11, 'General'), (12, 'General'), (13, 'General'), (14, 'General'), (15, 'General'), (16, 'General'), (17, 'General'), (18, 'General'), (19, 'General'), (20, 'General'), (21, 'General'), (22, 'General'), (23, 'General'), (24, 'General'), (25, 'General'), (26, 'General'), (27, 'General'), (28, 'General'), (29, 'General'), (30, 'General'), (31, 'General'), (32, 'General'), (33, 'General'), (34, 'General'), (35, 'General'), (36, 'General'), (37, 'General'), (38, 'General'), (39, 'General'), (40, 'General'), (41, 'General'), (42, 'General'), (43, 'General'), (44, 'General')]


In [11]:
# Load worksheet
wb = load_workbook(filename=new_file_path) # load the file path that's been created by the first script
ws = wb.active

### Currency/Date number formatting for cells

In [12]:
format_dictionary = {
    'currency':'"$"#,##0.00',
    'date':'d-mmm-yy'
    }

important_columns = {
    'Category': 'general', # this has to remain a default column
    'Date': 'date',
    'Description': 'general',
    'Net Donation': 'currency',
    'Stripe Fee': 'currency',
    'Platform Fee': 'currency',
    'Total Gross Donation': 'currency',
    'Event': 'general',
    'Source Title': 'general' 
                    }

# Loop through the ws columns
for col in ws.columns:
    column_name = col[0].value
    print(column_name)
    # Check if the column is in the important_columns dictionary
    if column_name in important_columns and important_columns[column_name] in format_dictionary:
        print('Formatting column', column_name)
        # Get the format from the dictionary
        col_format = format_dictionary[important_columns[column_name]]
        # Apply the format to the entire column
        for cell in col:
            cell.number_format = col_format

### Color formatting the cells

In [13]:
# Loop through the columns
for col in ws.columns:
    # Check if the column is in the columns_to_color list
    if col[0].value in important_columns and important_columns[col[0].value] == 'currency':
        for cell in col:
            number_value = cell.value
            if isinstance(number_value, (int, float)):
                if number_value < 0:
                    cell.font = Font(color='FF0000')

### Hide all columns that are not needed

In [14]:
# Loop through each column in the worksheet
for col in ws.columns:
    column_name = col[0].value
    if column_name not in important_columns:
        ws.column_dimensions[get_column_letter(col[0].column)].hidden = True

showing column: Date
Hiding column Time
showing column: Description
showing column: Net Donation
Hiding column Fees Covered
showing column: Stripe Fee
showing column: Platform Fee
showing column: Total Gross Donation
Hiding column Payment Type
Hiding column Brand
Hiding column Exp Month
Hiding column Exp Year
Hiding column Last4
Hiding column stripe_charge_id
Hiding column Business ID
Hiding column Business Name
Hiding column Email
showing column: Event
Hiding column Event ID
Hiding column Name
Hiding column Price IDs
Hiding column Price Name
Hiding column Source
showing column: Source Title
Hiding column customer_email
Hiding column customer_name
Hiding column order_id
Hiding column site_url
Hiding column charity_id
Hiding column charity_name
Hiding column donor_email
Hiding column fee_paid_by_donor
Hiding column first_name
Hiding column harness_id
Hiding column harness_platform
Hiding column honor_from
Hiding column honor_message
Hiding column honor_name
Hiding column last_name
Hidin

### Adding the categories and totals

In [15]:
# Add revenue categories and donation total

# Convert dictionary to list
category_map_list = []
for category in category_map:
  category_map_list.append([category, category_map[category]])
category_map_list.sort()

# Temporary variable assignment
spreadsheet_title = 'Payout Report for $123.45 on Mar 7 - asdfghjk'
# Temporary variable assignment

cells_to_convert_to_currency = []
label_column = ''
value_column = ''

# Figure out where to place the label and value columns
for col in ws.columns:
  if col[0].value in important_columns:
    if label_column == '':
      label_column = get_column_letter(col[0].column)
    else:
      value_column = get_column_letter(col[0].column)
      break

# Add the categories to the worksheet
row = ws.max_row + 3 # Start 3 rows below the last row
for category in category_map_list:
    label = category[0]
    value = category[1]
    ws[f"{label_column}{row}"] = label
    ws[f"{value_column}{row}"] = value
    cells_to_convert_to_currency.append(f"{value_column}{row}")
    row += 1
ws[f"{label_column}{row}"] = 'Total'
ws[f"{value_column}{row}"] = total_donations

# Add spreadsheet title with two buffer rows
row += 3
ws[f"{label_column}{row}"] = spreadsheet_title

# Loop through the cells and convert to currency
for cell in cells_to_convert_to_currency:
  ws[f"{cell}"].number_format = '"$"#,##0.00'

### Resize the column width to fit the data

In [16]:
default_column_widths = {
    'date': 20,
    'currency': 17,
    'general': 30
}

# Loop through the columns and set the width
for col in ws.columns:
    col_letter = get_column_letter(col[0].column)
    # If the column is in the important columns dictionary
    if col[0].value in important_columns:
        column_name = col[0].value
        width = default_column_widths[important_columns[column_name]]
        ws.column_dimensions[col_letter].width = width


### Save the file

In [17]:
# Save the workbook to a new file
wb.save(new_file_path)